Concurrency ve Paralel Programlama

THREADING:
Hafif, aynı süreç içinde hafıza paylaşımı (shared memory).

GIL nedeniyle CPU-bound işleri hızlandırmaz ama I/O-bound 
(ağ, disk, bekleme) işleri için uygundur.




In [2]:
import threading
import time
def take_cube(n):
    for i in n:
        print(i**3)
        time.sleep(0.20)

def take_square(n):
    for i in n:
        print(i**2)
        time.sleep(0.20)
nums=[1,2,3,4,5,6,7,8,9]

print("taking cubes and squares")
start=time.time()
take_cube(nums)
take_square(nums)
end=time.time()
print("time for both process:",float(end-start))



taking cubes and squares
1
8
27
64
125
216
343
512
729
1
4
9
16
25
36
49
64
81
time for both process: 3.6116414070129395


WITH THREADING

In [ ]:
thread_one=threading.Thread(target=take_cube,args=(nums,))
thread_two=threading.Thread(target=take_square,args=(nums,))

start=time.time()
thread_one.start()
thread_two.start()
end=time.time()                          #threadler devam ederken program calismaya devam etti bu yüzden arada bir yerde printledi
                                                          
print("time for both process:",float(end-start))

1
1
time for both process: 0.015722990036010742


8
4
27
9
64
16
125
25
216
36
343
49
64
512
81
729


In [4]:
thread_one=threading.Thread(target=take_cube,args=(nums,))
thread_two=threading.Thread(target=take_square,args=(nums,))

start=time.time()
thread_one.start()
thread_two.start()
thread_one.join()
thread_two.join()
end=time.time()                          #threadler bittikten sonra program ana akışına devam edecek
                                                          
print("time for both process:",float(end-start))

1
1
8
4
9
27
16
64
25
125
36216

49
343
512
64
729
81
time for both process: 1.8251495361328125


thread olmadan geçen süre:  3.611    thread işlemi ile geçen süre:1.809    

In [5]:
counter = 0                    #thread sürecini kilitleyerek diğer threadlerin yanlış zamanda işlemi yapıp yanlış sonuç çıkmasını engelliyoruz
lock = threading.Lock()                 # 1 Kilit oluştur
                                        #counter += 1 ifadesi Python’da tek adımda (atomik) çalışmaz. Aslında:
                                        # counter değerini oku
                                        # +1 ekle
                                        # Belleğe yaz
def worker(n):
    global counter
    for _ in range(n):               
        with lock:                      # 2️⃣ Kilidi al
            counter+= 1                 # 3️⃣ Kritik bölge (critical section)
                                        # 4️⃣ Kilit, blok bitince otomatik release edilir

threads = [threading.Thread(target=worker, args=(50,)) for _ in range(4)]    #4 farklı thread var 

for t in threads:
    t.start()

for t in threads:
    t.join()

print("Güvenli counter:", counter)

Güvenli counter: 200


MULTIPROCESSING:

In [6]:
import multiprocessing

def take_cube(n):
    for i in n:
        print(i**3)
        time.sleep(0.20)

def take_square(n):
    for i in n:
        print(i**2)
        time.sleep(0.20)
nums=[1,2,3,4,5,6,7,8,9]

if __name__=="__main__":
    p1=multiprocessing.Process(target=take_cube,args=(nums,))    #Eğer if __name__ == "__main__": kullanılmazsa:
    start=time.time()                                            #  Yeni süreç ana kodun tamamını tekrar çalıştırır.
    p1.start()                                                   #Bu sonsuz döngü, fazla işlem açılması, hatta çökme ile sonuçlanabilir.
    print("process is alive:",p1.is_alive())
    p1.join()
    print("time spent",time.time()-start)
    print("process is alive:",p1.is_alive())
    


process is alive: True
time spent 0.20665574073791504
process is alive: False


In [8]:
import asyncio
# async fonksiyonda await kullanmamız gerekir 

async def say_after(delay, msg):
    await asyncio.sleep(delay)  # non-blocking bekleme
    print(msg)

async def main():
    # ardışık çalışma
    await say_after(1, "hello")
    await say_after(1, "world")

await main()



hello
world
